In [2]:
from nltk.corpus import stopwords #you can remove stop words for speed
from nltk.cluster.util import cosine_distance
import numpy as np
import networkx as nx

In [3]:
file = open("text1.txt", "r")
#This file contains one paragraph of multiple sentences
filedata = file.readlines()
article = filedata[0].split(". ") #Just do the first paragraph
sentences = []
for sentence in article:
    print(sentence)
    sentences.append(sentence.replace("[^a-zA-Z]", " ").split(" "))

It was the best of times
It was the worst of times
It was the age of wisdom
It was the age of foolishness
What is the importance of age
This is the best example.


In [4]:
print("Sentences are ", sentences)

Sentences are  [['It', 'was', 'the', 'best', 'of', 'times'], ['It', 'was', 'the', 'worst', 'of', 'times'], ['It', 'was', 'the', 'age', 'of', 'wisdom'], ['It', 'was', 'the', 'age', 'of', 'foolishness'], ['What', 'is', 'the', 'importance', 'of', 'age'], ['This', 'is', 'the', 'best', 'example.']]


In [5]:
def sentence_similarity(sent1, sent2 ):
    sent1 = [w.lower() for w in sent1]
    sent2 = [w.lower() for w in sent2]
    all_words = list(set(sent1 + sent2))
    vector1 = [0] * len(all_words)
    vector2 = [0] * len(all_words)
    # build the vector for the first sentence
    for w in sent1:
        vector1[all_words.index(w)] += 1
    # build the vector for the second sentence
    for w in sent2:
        vector2[all_words.index(w)] += 1
    return 1 - cosine_distance(vector1, vector2)

In [6]:
similarity_matrix = np.zeros((len(sentences), len(sentences)))
for idx1 in range(len(sentences)):
    for idx2 in range(len(sentences)):
        if idx1 == idx2: #ignore if both are same sentences
            continue
        similarity_matrix[idx1][idx2] = sentence_similarity(sentences[idx1], sentences[idx2])
print("Smilarity matrix \n", similarity_matrix)

Smilarity matrix 
 [[0.         0.83333333 0.66666667 0.66666667 0.33333333 0.36514837]
 [0.83333333 0.         0.66666667 0.66666667 0.33333333 0.18257419]
 [0.66666667 0.66666667 0.         0.83333333 0.5        0.18257419]
 [0.66666667 0.66666667 0.83333333 0.         0.5        0.18257419]
 [0.33333333 0.33333333 0.5        0.5        0.         0.36514837]
 [0.36514837 0.18257419 0.18257419 0.18257419 0.36514837 0.        ]]


In [8]:
# Step 3 - Rank sentences in similarity martix
sentence_similarity_graph = nx.from_numpy_array(similarity_matrix)
scores = nx.pagerank(sentence_similarity_graph)
print("scores", scores)

scores {0: 0.19306449754902083, 1: 0.18095893645850156, 2: 0.1911855225552033, 3: 0.1911855225552033, 4: 0.14434636291527997, 5: 0.09925915796679063}


In [10]:
# Step 4 - Sort the rank and pick top sentences
ranked_sentence = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)
print("Indexes of top ranked_sentence order are \n\n",ranked_sentence)

Indexes of top ranked_sentence order are 

 [(0.19306449754902083, ['It', 'was', 'the', 'best', 'of', 'times']), (0.1911855225552033, ['It', 'was', 'the', 'age', 'of', 'wisdom']), (0.1911855225552033, ['It', 'was', 'the', 'age', 'of', 'foolishness']), (0.18095893645850156, ['It', 'was', 'the', 'worst', 'of', 'times']), (0.14434636291527997, ['What', 'is', 'the', 'importance', 'of', 'age']), (0.09925915796679063, ['This', 'is', 'the', 'best', 'example.'])]


In [15]:
#Step 5 - How many sentences to pick
n = int(input("How many sentences do you want in the summary? "))
#n=2
summarize_text = []
for i in range(n):
    summarize_text.append(" ".join(ranked_sentence[i][1]))
# Step 6 - Offcourse, output the summarize text
print("Summarize Text: \n", ". ".join(summarize_text))

How many sentences do you want in the summary? 3
Summarize Text: 
 It was the best of times. It was the age of wisdom. It was the age of foolishness
